## E.2 MLE for Gaussian AR(1)-GARCH(1,1)

Fit a Gaussian AR(1)-GARCH(1,1) to the 10-year government bond yield. Use the following procedure:

1. Write a function, called "garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, epsilon)". It takes the parameters of the variance equation as an input as well as the residuals of the mean equation. The function returns the GARCH(1,1) implied variance. Note, the first variance is computed using "epsilon[0]" and the start value of the variance, i.e. "sigma2_1". 

2. Write a second function, called, "Neg_loglikelihood_ar1_Garch11(parameters)". It takes the model parameters as input and returns the negative joint log likelihood function. 

3. Use smart starting values for the optimization (from last week's Python for Financial Data Science material, see below). In addition, we use rather uninformative starting values for beta and sigma2_1, namely 0.01 and 1, respectively. **Praktomat: estimated parameters from local unconstrained optimization**

4. You want to use a bounded optimizer to ensure the estimate imply: (i) stationary interest rates (stationary mean equation), (ii) positive unconditional interest rates, (iii) stationary variance of interest rates (stationary variance equation), (iv) positive unconditional variance of interest rate. **Type of optimizer: differential_evolution**. **Praktomat: estimated parameter global constrained optimization**

5. Hand-in the mathematical algorithm and pseudo code that underlines your Python implementation. 

**Data**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [2]:
y_df = pd.read_excel("GovBondYields.xls", index_col = [0])
y10_df= y_df[120]
y10_df


Date
1954-04-01    2.29
1954-05-01    2.37
1954-06-01    2.38
1954-07-01    2.30
1954-08-01    2.36
              ... 
2005-12-01    4.47
2006-01-01    4.42
2006-02-01    4.57
2006-03-01    4.72
2006-04-01    4.99
Name: 120, Length: 625, dtype: float64

In [3]:
T=len(y_df)
T

625

**LL**

$$
L_T(\phi_0, \phi_1, \alpha_0, \alpha_1, \beta_1, \sigma_1) = \prod_{t=2}^T \frac{1}{\sqrt{ 2 \pi (\alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1})}} \times \exp\left( -\frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})} \right)
$$

with $\sigma^2_t = \alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1}, s.t. \sigma^2_1 = \text{known parameter}$

Note:
 $$
 \ln (L_T(.)) = \sum_{t=2}^T -\frac{1}{2} \ln(2\pi [\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1}]) -  \frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})} 
 $$

**function GARCH_11_VARIANCE**

In [4]:
def garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, epsilon):
    sigma_square= np.zeros(623)
    sigma_square[0]= alpha_0 + alpha_1*epsilon[0]**2+ beta_1*sigma2_1
    for i in range(1,623):
        sigma_square[i]= alpha_0 + alpha_1*epsilon[i]**2+ beta_1*sigma_square[i-1]
    
    return sigma_square

**-ln(L_T)**

In [8]:
def Neg_loglikelihood_ar1_Garch11(parameters,r_t):
    phi_0   = parameters[0]
    phi_1   = parameters[1]
    alpha_0 = parameters[2]
    alpha_1 = parameters[3]
    beta_1  = parameters[4]
    sigma2_1 = parameters[5]

    means = phi_0 + phi_1 * r_t.iloc[:-1].values
    eps   = r_t.iloc[1:].values - means
    vars_  =  garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, eps[:-1])
       
    loglikeli = np.sum(-0.5 * np.log(2 * np.pi * vars_) - 0.5*(eps[1:])**2/(vars_))
    
    return -loglikeli
    

**Start Values from 2pass Estimation**

In [9]:
#start values for AR(1)-GARCH(1,1) parameters from last week's Python for Financial Data Science material
phi0_start = 0.0204
phi1_start = 0.9962
alpha0_start = 0.0004
alpha1_start = 0.3157
#uninformative start values for GARCH part
beta1_start = 0.01
sigma2_1_start = 1

ar1_garch11_params_start = [phi0_start, phi1_start, alpha0_start, alpha1_start, beta1_start, sigma2_1_start]

In [10]:
Neg_loglikelihood_ar1_Garch11(ar1_garch11_params_start, y10_df)


4290.227857000036

**Unconstrained Optimization: Nelder-Mead Optimization**

In [11]:
#min (-ln L_T)
import scipy.optimize as sco

#Non-Constrained, non-linear optimization, i.e. https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
ar1_arch1_params_optimal = sco.minimize(Neg_loglikelihood_ar1_Garch11, ar1_garch11_params_start, args=(y10_df,),
                                        method='Nelder-Mead', options={'disp': True})

<ipython-input-8-8ae57d30baeb>:13: RuntimeWarning: invalid value encountered in log
  loglikeli = np.sum(-0.5 * np.log(2 * np.pi * vars_) - 0.5*(eps[1:])**2/(vars_))


Optimization terminated successfully.
         Current function value: 166.840529
         Iterations: 651
         Function evaluations: 1042


In [12]:
ar1_arch1_params_optimal


 final_simplex: (array([[-9.40566161e-02,  1.02268374e+00,  6.73421893e-03,
         4.44696040e+00,  3.77398370e-03, -1.90019306e+01],
       [-9.40560653e-02,  1.02268361e+00,  6.73422438e-03,
         4.44696609e+00,  3.77396721e-03, -1.90019887e+01],
       [-9.40568899e-02,  1.02268378e+00,  6.73420528e-03,
         4.44695007e+00,  3.77400414e-03, -1.90018556e+01],
       [-9.40567120e-02,  1.02268375e+00,  6.73423164e-03,
         4.44696873e+00,  3.77398900e-03, -1.90019790e+01],
       [-9.40562154e-02,  1.02268363e+00,  6.73420848e-03,
         4.44695465e+00,  3.77398253e-03, -1.90019117e+01],
       [-9.40562086e-02,  1.02268371e+00,  6.73421511e-03,
         4.44695911e+00,  3.77396584e-03, -1.90019394e+01],
       [-9.40566746e-02,  1.02268374e+00,  6.73419686e-03,
         4.44694515e+00,  3.77399460e-03, -1.90018332e+01]]), array([166.84052934, 166.84052934, 166.84052934, 166.84052934,
       166.84052934, 166.84052935, 166.84052935]))
           fun: 166.84052933771687

**Output:**

**Stationary GARCH(1,1)** Stationary Conditions and Positivity Restrictions for the Variance:

Stationary mean equation:
$$
|\phi_1| < 1
$$

Stationary variance equation:
$$
\alpha_1 + \beta_1 < 1
$$

Positive unconditional variance forecast:
$$
\alpha_0 > 0 \qquad \text{and} \qquad \alpha_1, \beta_1, \sigma^2_{1} \in \mathcal{R}_+
$$

Positive unconditional interest rates:
$$
\phi_0 > 0, \qquad \phi_1 > 0
$$


**Bounded Optimization:** 

Hints:

1. Please specify the bounds for all the parameters. Please use 1 for all the upper bounds.

2. For inequality constraints please following the doc from scipy in the following link:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html. In addition, for $ \alpha_1 + \beta_1 < 1$ we specify it as $ 0<\alpha_1 + \beta_1 < 1$. 

3. Please use seed=1

In [38]:
 from scipy.optimize import NonlinearConstraint, differential_evolution, Bounds
   
 def constr_f(parameters):
        return np.array(parameters[3] + parameters[4])
    

nlc = NonlinearConstraint(constr_f, 0, 1)
    # (0,1)(0,1)(0,1)(0,1)(0,1)(0,1)
bound= Bounds(lb=[0,0,0,0,0,0], ub=[1,1,1,1,1,1])
    
bound_opt= differential_evolution(func=Neg_loglikelihood_ar1_Garch11, bounds=bound,args=(y10_df,),constraints=(nlc), seed=1)

In [39]:
bound_opt

           constr: [array([0.])]
 constr_violation: 0.0
              fun: -76.80268667513437
              jac: [array([[0., 0., 0., 1., 1., 0.]]), array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])]
            maxcv: 0.0
          message: 'Optimization terminated successfully.'
             nfev: 7004
              nit: 93
          success: True
                x: array([4.95223927e-02, 9.91773642e-01, 2.26752550e-04, 1.44879677e-01,
       8.55120296e-01, 2.80554606e-03])